In [1]:
import polars as pl
from datetime import datetime
import get_and_store as gas
import polars_indicators as pi

In [2]:
symbols = ['A','AA']

In [3]:
df = gas.get_symbols_df(symbols)
df = df.filter(pl.col('Date') >= datetime(2023, 1, 1))
df = df.collect()

In [4]:
df

Date,Open,High,Low,Close,Adj Close,Volume,Symbol
date,f64,f64,f64,f64,f64,i64,str
2023-01-03,151.960007,153.130005,148.470001,150.039993,150.039993,1414300,"""A"""
2023-01-04,151.649994,153.039993,150.240005,151.669998,151.669998,1247400,"""A"""
2023-01-05,150.0,153.070007,148.770004,152.110001,152.110001,1714600,"""A"""
2023-01-06,154.360001,154.639999,143.009995,147.669998,147.669998,2445000,"""A"""
2023-01-09,149.690002,151.279999,147.199997,147.470001,147.470001,1269600,"""A"""
2023-01-10,150.179993,155.550003,148.75,155.229996,155.229996,1565700,"""A"""
2023-01-11,155.229996,158.580002,155.229996,158.169998,158.169998,1241900,"""A"""
2023-01-12,158.160004,158.600006,155.580002,156.490005,156.490005,809100,"""A"""
2023-01-13,155.539993,157.279999,154.5,156.919998,156.919998,921700,"""A"""


In [5]:
ret1 = pi.simple_moving_average(df, 5)
ret2 = pi.simple_moving_average(ret1.df, 4)
ret2.df

Date,Open,High,Low,Close,Adj Close,Volume,Symbol,SMA5,SMA4
date,f64,f64,f64,f64,f64,i64,str,f64,f64
2023-01-03,151.960007,153.130005,148.470001,150.039993,150.039993,1414300,"""A""",null,null
2023-01-04,151.649994,153.039993,150.240005,151.669998,151.669998,1247400,"""A""",null,null
2023-01-05,150.0,153.070007,148.770004,152.110001,152.110001,1714600,"""A""",null,null
2023-01-06,154.360001,154.639999,143.009995,147.669998,147.669998,2445000,"""A""",null,150.372498
2023-01-09,149.690002,151.279999,147.199997,147.470001,147.470001,1269600,"""A""",149.791998,149.73
2023-01-10,150.179993,155.550003,148.75,155.229996,155.229996,1565700,"""A""",150.829999,150.619999
2023-01-11,155.229996,158.580002,155.229996,158.169998,158.169998,1241900,"""A""",152.129999,152.134998
2023-01-12,158.160004,158.600006,155.580002,156.490005,156.490005,809100,"""A""",153.006,154.34
2023-01-13,155.539993,157.279999,154.5,156.919998,156.919998,921700,"""A""",154.856,156.702499


In [6]:
ret3 = pi.crossover(ret2.df, ret1.column, ret2.column, direction='up')
ret3.df

Date,Open,High,Low,Close,Adj Close,Volume,Symbol,SMA5,SMA4,SMA5_cross_up_SMA4
date,f64,f64,f64,f64,f64,i64,str,f64,f64,bool
2023-01-03,151.960007,153.130005,148.470001,150.039993,150.039993,1414300,"""A""",null,null,false
2023-01-04,151.649994,153.039993,150.240005,151.669998,151.669998,1247400,"""A""",null,null,null
2023-01-05,150.0,153.070007,148.770004,152.110001,152.110001,1714600,"""A""",null,null,null
2023-01-06,154.360001,154.639999,143.009995,147.669998,147.669998,2445000,"""A""",null,150.372498,null
2023-01-09,149.690002,151.279999,147.199997,147.470001,147.470001,1269600,"""A""",149.791998,149.73,null
2023-01-10,150.179993,155.550003,148.75,155.229996,155.229996,1565700,"""A""",150.829999,150.619999,false
2023-01-11,155.229996,158.580002,155.229996,158.169998,158.169998,1241900,"""A""",152.129999,152.134998,false
2023-01-12,158.160004,158.600006,155.580002,156.490005,156.490005,809100,"""A""",153.006,154.34,false
2023-01-13,155.539993,157.279999,154.5,156.919998,156.919998,921700,"""A""",154.856,156.702499,false


In [7]:
ret3.df.filter(pl.col(ret3.column) == True)

Date,Open,High,Low,Close,Adj Close,Volume,Symbol,SMA5,SMA4,SMA5_cross_up_SMA4
date,f64,f64,f64,f64,f64,i64,str,f64,f64,bool
2023-01-18,156.259995,158.160004,153.639999,154.070007,154.070007,1150700,"""A""",156.210001,155.720001,true
2023-01-27,155.75,156.960007,154.809998,155.690002,155.690002,646200,"""A""",156.485999,155.842499,true
2023-02-07,151.490005,154.960007,151.130005,154.509995,154.509995,1126700,"""A""",154.45,154.200001,true
2023-02-13,153.270004,154.660004,152.009995,154.600006,154.600006,925400,"""A""",153.692001,153.487503,true
2023-02-16,148.809998,150.160004,147.229996,148.279999,148.279999,1101700,"""A""",152.044,151.9175,true
2023-03-01,138.580002,139.990005,136.25,137.509995,137.509995,3132800,"""A""",141.211996,140.714996,true
2023-03-09,141.259995,142.0,138.300003,138.929993,138.929993,1328900,"""A""",141.463995,140.847496,true
2023-01-19,51.139999,53.145,49.52,49.52,49.430809,12142700,"""AA""",53.152,53.110001,true
2023-01-30,51.91,52.130001,50.279999,50.34,50.249332,4370000,"""AA""",52.052,51.8475,true


In [8]:
ret3.df

Date,Open,High,Low,Close,Adj Close,Volume,Symbol,SMA5,SMA4,SMA5_cross_up_SMA4
date,f64,f64,f64,f64,f64,i64,str,f64,f64,bool
2023-01-03,151.960007,153.130005,148.470001,150.039993,150.039993,1414300,"""A""",null,null,false
2023-01-04,151.649994,153.039993,150.240005,151.669998,151.669998,1247400,"""A""",null,null,null
2023-01-05,150.0,153.070007,148.770004,152.110001,152.110001,1714600,"""A""",null,null,null
2023-01-06,154.360001,154.639999,143.009995,147.669998,147.669998,2445000,"""A""",null,150.372498,null
2023-01-09,149.690002,151.279999,147.199997,147.470001,147.470001,1269600,"""A""",149.791998,149.73,null
2023-01-10,150.179993,155.550003,148.75,155.229996,155.229996,1565700,"""A""",150.829999,150.619999,false
2023-01-11,155.229996,158.580002,155.229996,158.169998,158.169998,1241900,"""A""",152.129999,152.134998,false
2023-01-12,158.160004,158.600006,155.580002,156.490005,156.490005,809100,"""A""",153.006,154.34,false
2023-01-13,155.539993,157.279999,154.5,156.919998,156.919998,921700,"""A""",154.856,156.702499,false


In [9]:
index_name = 'ind'
column_name = 'test'
first_symbol = df[pi.SYMBOL_COLUMN].to_list()[0]
newdf = df.with_row_count(name=index_name).with_columns( 
                pl.when(
                    pl.col(pi.SYMBOL_COLUMN) == first_symbol) \
                    .then(pl.col(index_name) + 1) \
                .otherwise(
                    pl.col(index_name) - 1) \
                .alias(column_name))

In [10]:
newdf.columns

['ind',
 'Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Adj Close',
 'Volume',
 'Symbol',
 'test']

In [11]:
ret = pi.crossover(newdf, index_name, column_name)
ret.df.filter(pl.col(ret.column) == True)

ind,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,test,ind_cross_test
u32,date,f64,f64,f64,f64,f64,i64,str,i64,bool


In [12]:
df.filter(pl.col('Symbol') == 'B').is_empty()

True

In [16]:
lf = df.lazy()
lf

In [21]:
same_symbol = 'same_symbol'
cross_up = 'cross_up'
cross_down = 'cross_down'
SYMBOL_COLUMN = 'Symbol'
column1 = 'Open'
column2 = 'Close'
lf = lf.with_columns((pl.col(SYMBOL_COLUMN).shift(1) == pl.col(SYMBOL_COLUMN)).alias(same_symbol))


In [22]:
lf.collect()

Date,Open,High,Low,Close,Adj Close,Volume,Symbol,same_symbol
date,f64,f64,f64,f64,f64,i64,str,bool
2023-01-03,151.960007,153.130005,148.470001,150.039993,150.039993,1414300,"""A""",false
2023-01-04,151.649994,153.039993,150.240005,151.669998,151.669998,1247400,"""A""",true
2023-01-05,150.0,153.070007,148.770004,152.110001,152.110001,1714600,"""A""",true
2023-01-06,154.360001,154.639999,143.009995,147.669998,147.669998,2445000,"""A""",true
2023-01-09,149.690002,151.279999,147.199997,147.470001,147.470001,1269600,"""A""",true
2023-01-10,150.179993,155.550003,148.75,155.229996,155.229996,1565700,"""A""",true
2023-01-11,155.229996,158.580002,155.229996,158.169998,158.169998,1241900,"""A""",true
2023-01-12,158.160004,158.600006,155.580002,156.490005,156.490005,809100,"""A""",true
2023-01-13,155.539993,157.279999,154.5,156.919998,156.919998,921700,"""A""",true


In [23]:

lf = lf.with_columns(
        (
            (pl.col(column1) > pl.col(column2))
            &
            (pl.col(column1).shift(1) < pl.col(column2).shift(1))
        ).alias(cross_up)
    )

In [25]:
lf.collect()

Date,Open,High,Low,Close,Adj Close,Volume,Symbol,same_symbol,cross_up
date,f64,f64,f64,f64,f64,i64,str,bool,bool
2023-01-03,151.960007,153.130005,148.470001,150.039993,150.039993,1414300,"""A""",false,null
2023-01-04,151.649994,153.039993,150.240005,151.669998,151.669998,1247400,"""A""",true,false
2023-01-05,150.0,153.070007,148.770004,152.110001,152.110001,1714600,"""A""",true,false
2023-01-06,154.360001,154.639999,143.009995,147.669998,147.669998,2445000,"""A""",true,true
2023-01-09,149.690002,151.279999,147.199997,147.470001,147.470001,1269600,"""A""",true,false
2023-01-10,150.179993,155.550003,148.75,155.229996,155.229996,1565700,"""A""",true,false
2023-01-11,155.229996,158.580002,155.229996,158.169998,158.169998,1241900,"""A""",true,false
2023-01-12,158.160004,158.600006,155.580002,156.490005,156.490005,809100,"""A""",true,true
2023-01-13,155.539993,157.279999,154.5,156.919998,156.919998,921700,"""A""",true,false


In [26]:

lf = lf.with_columns(
    (
        (pl.col(column1) < pl.col(column2))
        &
        (pl.col(column1).shift(1) > pl.col(column2).shift(1))
    ).alias(cross_down)
)

In [27]:
lf.collect()

Date,Open,High,Low,Close,Adj Close,Volume,Symbol,same_symbol,cross_up,cross_down
date,f64,f64,f64,f64,f64,i64,str,bool,bool,bool
2023-01-03,151.960007,153.130005,148.470001,150.039993,150.039993,1414300,"""A""",false,null,false
2023-01-04,151.649994,153.039993,150.240005,151.669998,151.669998,1247400,"""A""",true,false,true
2023-01-05,150.0,153.070007,148.770004,152.110001,152.110001,1714600,"""A""",true,false,false
2023-01-06,154.360001,154.639999,143.009995,147.669998,147.669998,2445000,"""A""",true,true,false
2023-01-09,149.690002,151.279999,147.199997,147.470001,147.470001,1269600,"""A""",true,false,false
2023-01-10,150.179993,155.550003,148.75,155.229996,155.229996,1565700,"""A""",true,false,true
2023-01-11,155.229996,158.580002,155.229996,158.169998,158.169998,1241900,"""A""",true,false,false
2023-01-12,158.160004,158.600006,155.580002,156.490005,156.490005,809100,"""A""",true,true,false
2023-01-13,155.539993,157.279999,154.5,156.919998,156.919998,921700,"""A""",true,false,true


In [29]:
lf.with_columns(((pl.col(cross_up) | pl.col(cross_down)) & pl.col(same_symbol)).alias(column_name)).collect()

Date,Open,High,Low,Close,Adj Close,Volume,Symbol,same_symbol,cross_up,cross_down,test
date,f64,f64,f64,f64,f64,i64,str,bool,bool,bool,bool
2023-01-03,151.960007,153.130005,148.470001,150.039993,150.039993,1414300,"""A""",false,null,false,false
2023-01-04,151.649994,153.039993,150.240005,151.669998,151.669998,1247400,"""A""",true,false,true,true
2023-01-05,150.0,153.070007,148.770004,152.110001,152.110001,1714600,"""A""",true,false,false,false
2023-01-06,154.360001,154.639999,143.009995,147.669998,147.669998,2445000,"""A""",true,true,false,true
2023-01-09,149.690002,151.279999,147.199997,147.470001,147.470001,1269600,"""A""",true,false,false,false
2023-01-10,150.179993,155.550003,148.75,155.229996,155.229996,1565700,"""A""",true,false,true,true
2023-01-11,155.229996,158.580002,155.229996,158.169998,158.169998,1241900,"""A""",true,false,false,false
2023-01-12,158.160004,158.600006,155.580002,156.490005,156.490005,809100,"""A""",true,true,false,true
2023-01-13,155.539993,157.279999,154.5,156.919998,156.919998,921700,"""A""",true,false,true,true
